In [40]:
import time
import requests
import schedule
import pandas as pd
from bs4 import BeautifulSoup

In [15]:
# Find all articles in a single page

def extract_articles(page_url):
    data = requests.get(page_url)
    soup = BeautifulSoup(data.text)
    titles = soup.find_all('h3', class_='c-listing__title')
    title_item = []
    url_item = []

    for title in titles:
        article_title = title.a.get_text()
        article_url = title.a['href']
        title_item.append(article_title)
        url_item.append(article_url)
    
    return title_item, url_item

In [23]:
# Get the maximum page number

base_url = "https://jbiomedsci.biomedcentral.com/articles"
data = requests.get(base_url)
soup = BeautifulSoup(data.text)
pages = soup.find_all('a', class_ = "c-pagination__link")
max_page_number = 0
for page in pages:
    page_text = page.get_text().strip()
    if page_text.isdigit():
        page_num = int(page_text)
        if page_num > max_page_number:
            max_page_number = page_num

32


In [27]:
# Iterate all pages and get all the titles and urls

titles = []
urls = []

base_url = "https://jbiomedsci.biomedcentral.com/articles"

# Add base url titles and urls
title, url = extract_articles(base_url)
for t in title:
    titles.append(t)
for u in url:
    urls.append(u)

# Add second page to max page titles and urls
for i in range(2, max_page_number + 1):
    page_url = f"https://jbiomedsci.biomedcentral.com/articles?searchType=journalSearch&sort=PubDate&page={i}"
    title, url = extract_articles(page_url)
    for t in title:
        titles.append(t)
    for u in url:
        urls.append(u)

1563
1563


In [38]:
# Check the titles and urls length
assert(len(titles) == len(urls))

# Check if there's any duplicate urls inside the datasets
assert(len(urls) == len(set(urls)))

In [30]:
# Make pandas dataframe
df = pd.DataFrame({'Title':titles, 'URL':urls})
df.to_csv("articles.csv")